In [1]:
import pandas as pd
import numpy as np

In [2]:
MTURK_RES_DIR = "/Users/yiweiluo/mturk_tools/Submiterator/projectfiles/"

In [4]:
CURR_TASK = "cc_stance_LIVE_pilot_0/"

In [5]:
subject_info = pd.read_csv(MTURK_RES_DIR + CURR_TASK + "cc_stance_pilot_0-subject-info.csv")
trial_info = pd.read_csv(MTURK_RES_DIR + CURR_TASK + "cc_stance_pilot_0-trials.csv")

In [6]:
subject_info

,startDate,startTime,comments,criticisms,HitCorrect,HitFamiliar,age,gender,education,party,languages,endTime,timeSpent,likert1,likert2,likert3,likert4,likert5,likert6,workerid
0,Tue Oct 29 2019 12:50:13 GMT-0500 (Central Day...,1572371413516,"thanks for the opportunity, this was an intere...",NaN,0,1,22,male,graduated_college,republican,english,1572371987690,9.569567,strong_agree,agree,disagree,strong_agree,strong_disagree,strong_disagree,A2UREJJH126LX9
1,Tue Oct 29 2019 13:57:27 GMT-0400 (Eastern Day...,1572371847606,"Very difficult, more me, anyway.",none. thank you.,0,404,69,female,other,republican,english,1572372493079,10.757883,agree,strong_agree,strong_agree,strong_agree,neutral,strong_disagree,A2APG8MSLJ6G2K
2,Tue Oct 29 2019 14:00:24 GMT-0400 (Eastern Day...,1572372024960,no,none,0,1,39,female,graduated_high_school,democrat,"English, Arabic",1572372368903,5.732383,strong_agree,strong_agree,neutral,strong_agree,disagree,neutral,AMV1E7FFPVAW4
3,Tue Oct 29 2019 13:54:34 GMT-0400 (Eastern Day...,1572371674068,NaN,NaN,0,1,29,male,graduated_college,independent,English,1572372629479,15.923517,strong_agree,strong_agree,agree,strong_agree,agree,strong_agree,A2541C8MY0BYV3


# 1. Fleiss's kappa for 2+ annotators

In [7]:
trial_info.columns

Index(['trial_number', 'RT', 'condition', 'workerid', 'trial_sent_id',
       'option_chosen'],
      dtype='object')

Break trials into chunks for each annotator:

In [9]:
trials_per_worker = trial_info.groupby('workerid')

In [10]:
trials_per_worker.first()

,trial_number,RT,condition,trial_sent_id,option_chosen
workerid,,,,,
0,1,31346,true,t9,neutral
1,1,18869,screen,s2,neutral
2,1,17574,true,t14,agrees
3,1,6544,true,t12,neutral


In [21]:
worker_ids = list(trials_per_worker.first().index)

In [12]:
trials_per_worker.get_group(0)

,trial_number,RT,condition,trial_sent_id,option_chosen
0,1,31346,true,t9,neutral
1,2,5607,true,t4,agrees
2,3,4716,screen,s0,disagrees
3,4,21177,screen,s1,neutral
4,5,2778,screen,s3,agrees
5,6,5742,true,t19,disagrees
6,7,2465,true,t22,neutral
7,8,43071,true,t3,neutral
8,9,2266,screen,s4,neutral
9,10,3321,true,t17,disagrees


Step 1: Calculate how many assignments went (proportionally) to each category $(P_c)$.

$$P_c=\frac{1}{Aa}\sum_{i=1}^{A}a_{ic},$$

$A =$ number of annotators,
$a =$ number of annotations each annotator created,
$a_{ic} =$ number of annotations in category $c$ for annotator $i$ 

In [13]:
A = 4
a = 30

In [24]:
P_agree = 1./(A*a)*(np.sum([np.sum(trials_per_worker.get_group(w_id)['option_chosen'] == 'agrees')
                           for w_id in worker_ids]))
P_disagree = 1./(A*a)*(np.sum([np.sum(trials_per_worker.get_group(w_id)['option_chosen'] == 'disagrees')
                           for w_id in worker_ids]))
P_neutral = 1./(A*a)*(np.sum([np.sum(trials_per_worker.get_group(w_id)['option_chosen'] == 'neutral')
                           for w_id in worker_ids]))

In [26]:
P_neutral

0.475

Step 2: Calculate each annotator's agreement with other annotators, compared to all possible agreement values ($P_i$):

$$P_i = \frac{(\sum_{c=1}^ka_{ic}^2)-a}{a(a-1)},$$

$k =$ number of categories.

In [27]:
k = 3

In [29]:
P_annotators = {}
for w_id in worker_ids:
    P_annotators[w_id] = (np.sum(trials_per_worker.get_group(w_id)['option_chosen'] == 'agrees')**2+\
    np.sum(trials_per_worker.get_group(w_id)['option_chosen'] == 'disagrees')**2+\
    np.sum(trials_per_worker.get_group(w_id)['option_chosen'] == 'neutral')**2-a)/(a*(a-1.))

In [30]:
P_annotators

{0: 0.45057471264367815,
 1: 0.3195402298850575,
 2: 0.37471264367816093,
 3: 0.3724137931034483}

Step 3: Calculate $\bar{P}$, the average of ```P_annotators```:

In [31]:
P_av = np.mean(list(P_annotators.values()))

In [32]:
P_av

0.37931034482758624

Step 4: Calculate $\bar{P_e}$, the average of the expected agreement:

In [33]:
P_e = np.sum(P_agree**2 + P_disagree**2 + P_neutral**2)

In [34]:
P_e

0.36930555555555555

Step 5: Calculate Fleiss's kappa:

$$K = \frac{\bar{P}-\bar{P_e}}{1-\bar{P_e}}$$

In [35]:
kappa = (P_av-P_e)/(1.-P_e)

In [36]:
kappa

0.01586313207633142

"Slight agreement" within the range 0.01 - 0.20.

# 2. Correlation

Fleiss's kappa only measures agreement over category totals. Looking at correlation also gives us a p-value.

In [62]:
labels_per_worker = {}
for w_id in worker_ids:
    worker_labels = trials_per_worker.get_group(w_id).sort_values(by='trial_sent_id')
    worker_labels = worker_labels.set_index('trial_sent_id')
    worker_labels = worker_labels.drop(["trial_number","RT","condition"],axis=1)
    worker_labels.columns = ['worker_{}'.format(w_id)]
    labels_per_worker[w_id] = worker_labels

In [67]:
all_worker_labels = pd.concat([labels_per_worker[0], labels_per_worker[1], labels_per_worker[2], 
                               labels_per_worker[3]], axis=1)

In [68]:
all_worker_labels

,worker_0,worker_1,worker_2,worker_3
trial_sent_id,,,,
s0,disagrees,disagrees,disagrees,disagrees
s1,neutral,neutral,neutral,agrees
s2,neutral,neutral,neutral,neutral
s3,agrees,agrees,agrees,agrees
s4,neutral,neutral,neutral,neutral
t0,neutral,agrees,neutral,neutral
t1,agrees,agrees,agrees,agrees
t10,neutral,agrees,neutral,neutral
t11,neutral,agrees,neutral,neutral


Step 1: Make binarized matrices for each annotator.

In [74]:
def convert_int(l):
    return int(l)

In [83]:
worker_matrices = {}

In [86]:
for w_id in worker_ids:
    label_matrix_w = pd.DataFrame.from_dict({'trial_sent_id':all_worker_labels.index,
                                        'agrees':list((all_worker_labels['worker_{}'.format(w_id)] == 'agrees').apply(convert_int)),
                                        'disagrees':list((all_worker_labels['worker_{}'.format(w_id)] == 'disagrees').apply(convert_int)),
                                        'neutral':list((all_worker_labels['worker_{}'.format(w_id)] == 'neutral').apply(convert_int))})
    label_matrix_w = label_matrix_w.set_index('trial_sent_id')
    worker_matrices[w_id] = label_matrix_w

In [90]:
worker_matrices[3]

,agrees,disagrees,neutral
trial_sent_id,,,
s0,0,1,0
s1,1,0,0
s2,0,0,1
s3,1,0,0
s4,0,0,1
t0,0,0,1
t1,1,0,0
t10,0,0,1
t11,0,0,1
